In [ ]:
#참고 : https://bigdata-doctrine.tistory.com/34

In [45]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [24]:
def ex_tag(sid, page):
    ### 뉴스 분야(sid)와 페이지(page)를 입력하면 그에 대한 링크들을 리스트로 추출하는 함수 ###
    
    ## sid -> 100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학
    url = f"https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={sid}"\
    "#&date=%2000:00:00&page={page}"
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    a_tag = soup.find_all("a")
    
    ## 기사 사이트 
    tag_lst = []
    for a in a_tag:
        if "href" in a.attrs:  # href가 있는것만 고르는 것
            if (f"sid={sid}" in a["href"]) and ("article" in a["href"]):
                tag_lst.append(a["href"])
                
    return tag_lst

In [29]:
def re_tag(sid):
    ### 특정 분야의 100페이지까지의 뉴스의 링크를 수집하여 중복 제거한 리스트로 변환하는 함수 ###
    re_lst = []
    for i in tqdm(range(100)):
        lst = ex_tag(sid, i+1)
        re_lst.extend(lst)

    # 중복 제거
    re_set = set(re_lst)
    re_lst = list(re_set)
    
    return re_lst

In [30]:
all_hrefs = {}
sids = [i for i in range(100,106)]  # 분야 리스트

# 각 분야별로 링크 수집해서 딕셔너리에 저장
for sid in sids:
    sid_data = re_tag(sid)
    all_hrefs[sid] = sid_data

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [31]:
def art_crawl(all_hrefs, sid, index):
    """
    sid와 링크 인덱스를 넣으면 기사제목, 날짜, 본문을 크롤링하여 딕셔너리를 출력하는 함수 
    
    Args: 
        all_hrefs(dict): 각 분야별로 100페이지까지 링크를 수집한 딕셔너리 (key: 분야(sid), value: 링크)
        sid(int): 분야 [100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학]
        index(int): 링크의 인덱스
    
    Returns:
        dict: 기사제목, 날짜, 본문이 크롤링된 딕셔너리
    
    """
    art_dic = {}
    
    ## 1.
    title_selector = "#title_area > span"
    date_selector = "#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans"\
    "> div.media_end_head_info_datestamp > div:nth-child(1) > span"
    main_selector = "#dic_area"
    
    url = all_hrefs[sid][index]
    html = requests.get(url, headers = {"User-Agent": "Mozilla/5.0 "\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    
    ## 2.
    # 제목 수집
    title = soup.select(title_selector)
    title_lst = [t.text for t in title]
    title_str = "".join(title_lst)
    
    # 날짜 수집
    date = soup.select(date_selector)
    date_lst = [d.text for d in date]
    date_str = "".join(date_lst)
    
    # 본문 수집
    main = soup.select(main_selector)
    main_lst = []
    for m in main:
        m_text = m.text
        m_text = m_text.strip()
        main_lst.append(m_text)
    main_str = "".join(main_lst)
    
    ## 3.
    art_dic["title"] = title_str
    art_dic["date"] = date_str
    art_dic["main"] = main_str
    
    return art_dic

In [32]:
# 모든 섹션의 데이터 수집 (제목, 날짜, 본문, section, url)
section_lst = [s for s in range(100, 106)]
artdic_lst = []

for section in tqdm(section_lst):
    for i in tqdm(range(len(all_hrefs[section]))):
        art_dic = art_crawl(all_hrefs, section, i)
        art_dic["section"] = section
        art_dic["url"] = all_hrefs[section][i]
        artdic_lst.append(art_dic)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/442 [00:00<?, ?it/s]

  0%|          | 0/571 [00:00<?, ?it/s]

  0%|          | 0/628 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

  0%|          | 0/266 [00:00<?, ?it/s]

  0%|          | 0/259 [00:00<?, ?it/s]

In [116]:
import pandas as pd

art_df = pd.DataFrame(artdic_lst)

In [117]:
# 100: 정치, 101: 경제, 102: 사회, 103: 생활/문화, 104: 세계, 105: IT/과학
def sec_chan(sec):
    if sec == 100:
        return '정치'
    elif sec == 101:
        return '경제'
    elif sec == 102:
        return '사회'
    elif sec == 103:
        return '생활/문화'
    elif sec == 104:
        return '세계'
    elif sec == 105:
        return 'IT/과학'

In [118]:
art_df['section'] = list(map(sec_chan, art_df['section']))

In [119]:
# 출력 옵션 설정
# pd.set_option("display.max_rows", None)  # 모든 행 표시
pd.set_option("display.max_columns", None)  # 모든 열 표시
pd.set_option("display.width", None)  # 줄 바꿈 없이 전체 내용 표시

In [120]:
art_df['main'][0]

'기존 수출금융 41조원에 22조원 추가 \n\n\n\n발언하는 박대출 정책위의장 (서울=연합뉴스) 하사헌 기자 = 국민의힘 박대출 정책위의장이 10일 오전 서울 여의도 국회에서 열린 \'수출 경쟁력 강화를 위한 수출금융 종합지원대책 민당정협의회\'에서 발언하고 있다. 2023.8.10 toadboy@yna.co.kr (끝) [파이낸셜뉴스] 국민의힘과 정부가 10일 수출기업의 경쟁력 강화를 위해 총 63조원의 수출금융을 지원하기로 뜻을 모았다. 기존 수출금융 41조원에 새롭게 22조원이 추가되는 방식이다.   당정은 이날 국회에서 수출 경쟁력 강화를 위한 수출금융 종합지원 대책 논의 민당정 협의회를 갖고 이같이 결정했다. 박대출 국민의힘 정책위의장은 "오늘 민당정 협의회는 비상이 걸린 수출 돌파구를 마련하는 자리"라며 "수출기업의 경쟁력 강화를 위해 현재 수출금융 41조원에 신규로 22조원을 추가해 총 63조원 규모를 공급하기로 했다"고 설명했다.   추가로 공급되는 22조원은 새로운 수출활로 개척 지원에 최소 4조1000억원, 수출전략산업 경쟁력 강화에 17조9000억원이 각각 지원될 예정이다.   민간과 공공 부문이 협력해 수출금융 지원에도 적극 나서기로 했다. 박 정책위의장은 "그동안 정부가 발표한 수출지원 대책은 예산·정책금융기관의 저리대출, 공공부문의 재원을 바탕으로 했다면 이번 대책은 정책금융기관과 민간의 힘을 합쳐 재원을 지원하는 역할을 하도록 해 은행의 역할을 강화하도록 한 것이 특징"이라고 설명했다.   가령, 대기업과 동반해 해외에 진출하는 협력업체에 대해서는 공적보증기관과 은행이 협업해 1조원 규모의 수출금융을 지원할 계획이다. 이에 따라 현대자동차가 해외 전기차 공장을 건설할 때 협력업체가 같이 진출할 수 있도록 현대차, 신용보증기금, 은행 등이 협력해서 필요한 자금 3000억원을 지원하기로 했다.   아울러 당정은 해외 프로젝트 패키지 금융모델을 구축해 해외의 대규모 프로젝트 수주를 지원하기로 했다. 이를 위해 산업은행, 수출입은행, 한

In [121]:
# 텍스트 전처리
def text_preprocessing(text):
    text = text.replace('\n', '')
    text = text.replace('   ', ' ')
    text = text.replace('\\', '')
    return text

In [122]:
art_df['main'] = art_df['main'].apply(text_preprocessing)

In [145]:
from datetime import datetime

def to_dt_time(date):
    date = datetime.strptime(date[:10], '%Y.%m.%d')
    date = date.strftime('%Y-%m-%d')
    return date

In [158]:
# 어제 뉴스만 사용(어제로 바꾸면 될듯?)
now = datetime.now()
now = now.date().strftime('%Y-%m-%d')

art_df['ymd'] = art_df['date'].apply(to_dt_time)

In [162]:
art_df[art_df['ymd'] == now]

,title,date,main,section,url,ymd
0,"당정, 수출금융 63조원 지원해 수출기업 경쟁력 강화 나선다",2023.08.10. 오후 1:31,기존 수출금융 41조원에 22조원 추가 발언하는 박대출 정책위의장 (서울=연합뉴스)...,정치,https://n.news.naver.com/mnews/article/014/000...,2023-08-10
1,"[자막뉴스] ""중요 군사행동 지침 시달""...서울 콕 짚은 北 김정은",2023.08.10. 오후 3:48,김정은 북한 국무위원장이 당 중앙군사위원회 확대회의에서 '공세적인 전쟁 준비 태세'...,정치,https://n.news.naver.com/mnews/article/052/000...,2023-08-10
3,김태우 사면 대상 포함…與 강서구청장 '무공천' 기류 변할까,2023.08.10. 오전 12:01,대법 선고 3개월 만에 전격 사면尹 '김태우는 공익제보자' 인정한 셈국민의힘 보궐 ...,정치,https://n.news.naver.com/mnews/article/119/000...,2023-08-10
4,"통일부, 北 '전쟁 준비' 언급에 ""적반하장…개탄스럽다""",2023.08.10. 오전 11:21,"""北 전쟁 준비 나설수록 강력한 한미 확장억제 직면할 것""열병식 예고는 ""이례적"" ...",정치,https://n.news.naver.com/mnews/article/421/000...,2023-08-10
5,"탁현민 “BTS가 데리고 다닌다고 끌려가겠느냐, 모르면 그냥 놔둬라”",2023.08.10. 오후 2:30,그룹 방탄소년단(BTS). [팬 커뮤니티 위버스][헤럴드경제=이원율 기자]문재인 정...,정치,https://n.news.naver.com/mnews/article/016/000...,2023-08-10
...,...,...,...,...,...,...
2220,"SKT-독립기념관, AI로 유관순 열사 등 독립운동사 영상 재현",2023.08.10. 오전 9:01,AI 이미지 복원기술 슈퍼노바 적용…광복절 앞두고 독립기념관서 공개(서울=연합뉴스)...,IT/과학,https://n.news.naver.com/mnews/article/001/001...,2023-08-10
2222,"네오위즈, 2분기 영업손 49억…""P의 거짓 출시 기대""",2023.08.10. 오전 9:26,"매출 701억원·순이익 224억원""다수 신작 출시로 하반기 반등 자신""네오위즈 20...",IT/과학,https://n.news.naver.com/mnews/article/119/000...,2023-08-10
2225,"""실물 이렇구나""…AI로 복원한 안중근·유관순 모습 보니 [영상]",2023.08.10. 오전 10:28,SK텔레콤 AI 이미지 복원기술 '슈퍼노바' 적용지금까지 흐릿한 사진으로만 남아있는...,IT/과학,https://n.news.naver.com/mnews/article/015/000...,2023-08-10
2226,"방통위, 정연주 방심위원장에 경고 조치… “업무추진비 선결제, 근태도 불량”",2023.08.10. 오전 10:42,방송통신심의위원회 현판(방심위 제공)방송통신위원회는 10일 방송통신심의위원회 국고보...,IT/과학,https://n.news.naver.com/mnews/article/020/000...,2023-08-10


In [163]:
art_df = art_df[art_df['ymd'] == now]

In [164]:
art_df.to_csv("article_df.csv")